# Importação das Bibliotecas

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Definição de Parâmetros

In [5]:
epochs = 100
learning_rate = 0.01

# Arquitetura da Rede Neural

In [6]:
class COVID_predictor(nn.Module):
    def __init__(self):
        super(COVID_predictor, self).__init__()
        self.fc1 = nn.Linear(1, 16)
        self.fc2 = nn.Linear(16, 16)
        self.output = nn.Linear(16, 1)
        
        self.dropout = nn.Dropout(p=0.2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = torch.sigmoid(self.output(x))
        return x


# Instanciando Custo e Otimizador

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = COVID_predictor().to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Cálculo da Acurácia

In [8]:
def accuracy(outputs, labels):
    preds = outputs.round()  # Arredonda para 0 ou 1 (binário)
    return (preds == labels).sum().item() / len(labels)

# Funções de Treinamento e Teste

In [9]:
def train(model, device, X_train, y_train, criterion, optimizer, epoch):
    model.train()
    inputs = torch.tensor(X_train, dtype=torch.float32).to(device)
    labels = torch.tensor(y_train, dtype=torch.float32).to(device)
    
    outputs = model(inputs)
    loss = criterion(outputs, labels.unsqueeze(1))
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Calcular acurácia
    acc = accuracy(outputs, labels.unsqueeze(1))
    
    if (epoch + 1) % 10 == 0:
        print()
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}, Accuracy: {acc:.4f}')
        print()
        
def test(model, device, X_test, y_test, criterion):
    model.eval()
    with torch.no_grad():
        inputs = torch.tensor(X_test, dtype=torch.float32).to(device)
        labels = torch.tensor(y_test, dtype=torch.float32).to(device)
        
        outputs = model(inputs)
        loss = criterion(outputs, labels.unsqueeze(1))
        acc = accuracy(outputs, labels.unsqueeze(1))
        
    print(f'Test Loss: {loss.item():.4f}, Test Accuracy: {acc:.4f}')



# Execução dos Treinamentos e dos Testes

In [10]:
for epoch in range(epochs):
    train(model, device, x_train, y_train, criterion, optimizer, epoch)
    test(model, device, x_test, y_test, criterion)

# Visualizando os Resultados

In [ ]:
num_samples = 10
inputs = torch.tensor(x_test[:num_samples], dtype=torch.float32).to(device)
indices = x_test[:num_samples].flatten()
true_labels = y_test[:num_samples]

with torch.no_grad():
    outputs = model(inputs)
    preds = outputs.round().flatten()

results = pd.DataFrame({
    'Índice': indices,
    'Predição': preds,
    'Label': true_labels
})

print()
print(results.to_string(index=False, float_format="%.2f"))